# **Fake News Detection - Method 2 (Testing)**

Ignore warnings

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

Constants and hyperparameters

In [2]:
from pathlib import Path


DATASETS = ["ISOT", "LIAR"]
SAVED_MODELS = Path("saved_models/method2")
MAX_LEN = 250
PARAMS = [(100, 5),(100, 8),(200, 5),(200, 8)]

Function to load method 1 features

In [3]:
import joblib
from scipy import sparse


def load_aux(ds, df):
    obj = joblib.load({
        "ISOT": "data/features/ISOT/bow_min30_chi2700.joblib",
        "LIAR": "data/features/LIAR/tfidf_min40_chi2700.joblib"}[ds])
    X = obj["vect"].transform(df)
    return obj["chi2"].transform(X)

Calculate model's(Bi-LSTM's) performance

In [4]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import models  # type: ignore


for ds in DATASETS:
    test_df = joblib.load(f"data/processed/{ds}/{ds.lower()}_test.pkl")
    aux_X = load_aux(ds, test_df["cleaned"].values)
    y_true = test_df["label"].values
    
    tok = joblib.load(SAVED_MODELS / "tokenizer" / f"{ds}.pkl")
    seqs = pad_sequences(tok.texts_to_sequences(test_df["cleaned"]), maxlen=MAX_LEN, padding="post", truncating="post")
    
    for dim, win in PARAMS:
        tag = f"{ds}_{dim}d_win{win}"
        model_path = SAVED_MODELS / "bilstm" / f"{tag}.keras"
        if not model_path.exists():
            print(f"Missing model: {model_path.name}")
            continue
        
        model = models.load_model(model_path, compile=False)
        y_pred_prob = model.predict([seqs, aux_X], batch_size=512, verbose=0).ravel()
        y_pred = (y_pred_prob >= 0.5).astype(int)
        
        print(f"\nResults on: dataset={ds}, w2v_dim={dim}, w2v_window_size={win} model=Bi-LSTM")
        print(classification_report(y_true, y_pred, digits=4))
        
        print("Confusion matrix:")
        print(confusion_matrix(y_true, y_pred))

E0000 00:00:1753559221.861059   30507 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753559221.864656   30507 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753559221.873492   30507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753559221.873509   30507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753559221.873511   30507 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753559221.873512   30507 computation_placer.cc:177] computation placer already registered. Please check linka


Results on: dataset=ISOT, w2v_dim=100, w2v_window_size=5 model=Bi-LSTM
              precision    recall  f1-score   support

           0     0.9979    0.9991    0.9985      4696
           1     0.9991    0.9977    0.9984      4284

    accuracy                         0.9984      8980
   macro avg     0.9985    0.9984    0.9984      8980
weighted avg     0.9984    0.9984    0.9984      8980

Confusion matrix:
[[4692    4]
 [  10 4274]]

Results on: dataset=ISOT, w2v_dim=100, w2v_window_size=8 model=Bi-LSTM
              precision    recall  f1-score   support

           0     0.9979    0.9991    0.9985      4696
           1     0.9991    0.9977    0.9984      4284

    accuracy                         0.9984      8980
   macro avg     0.9985    0.9984    0.9984      8980
weighted avg     0.9984    0.9984    0.9984      8980

Confusion matrix:
[[4692    4]
 [  10 4274]]

Results on: dataset=ISOT, w2v_dim=200, w2v_window_size=5 model=Bi-LSTM
              precision    recall  f1-sc